Connected to Google drive to load the Preprocced data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Clone the git repo

In [3]:
!git clone https://github.com/nlpyang/BertSum.git

Cloning into 'BertSum'...
remote: Enumerating objects: 301, done.
remote: Total 301 (delta 0), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (301/301), 15.03 MiB | 4.44 MiB/s, done.
Resolving deltas: 100% (174/174), done.


In [4]:
%cd BertSum/src

/content/BertSum/src


In [30]:
!pip install pytorch_pretrained_bert
!pip install tensorboardX
!pip install pyrouge
!pip install stanfordnlp
import stanfordnlp

Before training the model with the preprocessed data, I had to do the following changes in /src/models/data_loader.py :
1. Line no. 31 - Change the line from  "mask = 1 - (src == 0)" to "mask = ~(src == 0)" 
2. Line no. 34 - Change the line from "mask_cls = 1 - (clss == -1)" to "mask_cls = ~(clss == -1)"
3. Add a new line after Line no.220 - " p_batch = [x for x in p_batch if x != []] "

Give the argument bert_data_path to the preprocessed data and change the other arguments if required.


In [28]:
!python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path '/content/gdrive/My Drive/BertSum/BCP/bert_data' -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 100 -batch_size 20 -decay_method noam -train_steps 120 -accum_count 2 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 10

[2020-01-23 09:23:04,466 INFO] Device ID 0
[2020-01-23 09:23:04,466 INFO] Device cuda
[2020-01-23 09:23:05,411 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2020-01-23 09:23:05,411 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpt7tiendf
[2020-01-23 09:23:09,078 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-01-23 09:23:14,805 INFO] Summarizer(
  (bert): 

Now, in the validation mode, provide the path to the data to be validated . It will load the model which was saved in the previous step and validate the data. At the end, there will be some error related to Pyrouge as I couldn't install it, but the results will be saved in the BertSum/results folder

In [42]:
!python train.py -mode validate -bert_data_path '/content/gdrive/My Drive/BertSum/BCP/bert_data' -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 30000  -log_file LOG_FILE  -result_path ../results/cnndm -test_all -block_trigram true

[2020-01-23 09:40:12,104 INFO] Loading checkpoint from ../models/bert_classifier/model_step_100.pt
Namespace(accum_count=1, batch_size=30000, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/gdrive/My Drive/BertSum/BCP/bert_data', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='LOG_FILE', lr=1, max_grad_norm=0, mode='validate', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='../temp', test_all=True, test_from='', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2020-01-23 09:40:18,479 INFO] Loading valid dataset from /content/gdrive/My Drive/BertSum/BCP/bert_data.valid.pt, number 